In [24]:
# Import Dependencies
import os
import requests 
import json
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from pprint import pprint

# File to load
spotify_df = pd.read_csv("Resources/Data Set under 100mb.csv") 

spotify_df.head()

,name,popularity,duration_ms,explicit,artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,Carve,6,126903,0,['Uli'],2/22/1922,0.645,0.4450,0,-13.338,1,0.4510,0.674,0.7440,0.151,0.127,104.851,3
1,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],6/1/1922,0.695,0.2630,0,-22.136,1,0.9570,0.797,0.0000,0.148,0.655,102.009,1
2,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],3/21/1922,0.434,0.1770,1,-21.180,1,0.0512,0.994,0.0218,0.212,0.457,130.418,5
3,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],3/21/1922,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.9180,0.104,0.397,169.980,3
4,Lady of the Evening,0,163080,0,['Dick Haymes'],1922,0.402,0.1580,3,-16.900,0,0.0390,0.989,0.1300,0.311,0.196,103.220,4


In [26]:
multiVariable_df = spotify_df.drop(spotify_df.columns[[3,4,5,10,17]], axis = 1)
multiVariable_df.head()

,name,popularity,duration_ms,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,Carve,6,126903,0.645,0.4450,0,-13.338,0.4510,0.674,0.7440,0.151,0.127,104.851
1,Capítulo 2.16 - Banquero Anarquista,0,98200,0.695,0.2630,0,-22.136,0.9570,0.797,0.0000,0.148,0.655,102.009
2,Vivo para Quererte - Remasterizado,0,181640,0.434,0.1770,1,-21.180,0.0512,0.994,0.0218,0.212,0.457,130.418
3,El Prisionero - Remasterizado,0,176907,0.321,0.0946,7,-27.961,0.0504,0.995,0.9180,0.104,0.397,169.980
4,Lady of the Evening,0,163080,0.402,0.1580,3,-16.900,0.0390,0.989,0.1300,0.311,0.196,103.220


In [27]:
x = multiVariable_df.drop('popularity', axis = 1)
y = ['popularity']